# 6.2 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)
**토픽 모델링은 문서의 집합에서 토픽을 찾아내는 프로세스**를 말합니다. 이는 검색 엔진, 고객 민원 시스템 등과 같이 **문서의 주제를 알아내는** 일이 중요한 곳에서 사용됩니다. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)은 토픽 모델링의 대표적인 알고리즘입니다. 줄여서 LDA라고 합니다.

LDA는 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정합니다. 데이터가 주어지면, LDA는 문서가 생성되던 과정을 역추적합니다.

참고 링크 : https://lettier.com/projects/lda-topic-modeling/

위의 사이트는 코드 작성 없이 입력한 문서들로부터 DTM을 만들고 LDA를 수행한 결과를 보여주는 웹 사이트입니다.

### 1. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA) 개요
우선 LDA의 내부 메커니즘에 대해서 이해하기 전에, LDA를 일종의 블랙 박스로 보고 LDA에 문서 집합을 입력하면, 어떤 결과를 보여주는지 간소화 된 예를 들어 보겠습니다. 아래와 같은 3개의 문서가 있다고 합시다. 지금의 예제는 간단해서 눈으로도 토픽 모델링을 할 수 있을 것 같지만, 실제 수십만개 이상의 문서가 있는 경우는 직접 토픽을 찾아내는 것이 어렵기 때문에 LDA의 도움이 필요합니다.
```
문서1 : 저는 사과랑 바나나를 먹어요
문서2 : 우리는 귀여운 강아지가 좋아요
문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요
```
LDA를 수행할 때 

1. 문서 집합에서 토픽이 몇 개가 존재할지 가정하는 것은 사용자가 해야 할 일입니다. 
2. 여기서는 LDA에 2개의 토픽을 찾으라고 요청하겠습니다. 
3. 토픽의 개수를 의미하는 변수를 k라고 하였을 때, k를 2로 한다는 의미입니다. 
4. k의 값을 잘못 선택하면 원치않는 이상한 결과가 나올 수 있습니다. 

이렇게 모델의 성능에 영향을 주는 사용자가 직접 선택하는 매개변수를 머신 러닝 용어로 **하이퍼파라미터**라고 합니다. 이러한 하이퍼파라미터의 선택은 여러 실험을 통해 얻은 값일 수도 있고, 우선 시도해보는 값일 수도 있습니다.

LDA가 위의 세 문서로부터 2개의 토픽을 찾은 결과는 아래와 같습니다. 여기서는 LDA 입력 전에 주어와 불필요한 조사 등을 제거하는 전처리 과정은 거쳤다고 가정합니다. 즉, 전처리 과정을 거친 DTM이 LDA의 입력이 되었다고 가정합니다.

LDA는 각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정합니다.
```
<각 문서의 토픽 분포>
문서1 : 토픽 A 100%
문서2 : 토픽 B 100%
문서3 : 토픽 B 60%, 토픽 A 40%
```
```
<각 토픽의 단어 분포>
토픽A : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜찍하고 0%, 좋아요 0%
토픽B : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%
```
LDA는 토픽의 제목을 정해주지 않지만, 이 시점에서 알고리즘의 사용자는 위 결과로부터 두 토픽이 각각 과일에 대한 토픽과 강아지에 대한 토픽이라고 판단해볼 수 있습니다. 이제 LDA에 대해서 알아봅시다. 실제로 LDA는 아래의 설명보다 훨씬 더 복잡하지만, 여기서는 수학적인 수식은 배제하고 개념적 이해에 초점을 둡니다.

### 2. LDA의 가정
LDA는 문서의 집합으로부터 어떤 토픽이 존재하는지를 알아내기 위한 알고리즘입니다. LDA는 앞서 배운 빈도수 기반의 표현 방법인 BoW의 행렬 DTM 또는 TF-IDF 행렬을 입력으로 하는데, 이로부터 알 수 있는 사실은 LDA는 단어의 순서는 신경쓰지 않겠다는 겁니다.

LDA는 문서들로부터 토픽을 뽑아내기 위해서 이러한 가정을 염두해두고 있습니다. 모든 문서 하나, 하나가 작성될 때 그 문서의 작성자는 이러한 생각을 했습니다. '나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야.' 조금 더 구체적으로 알아보겠습니다. 각각의 문서는 다음과 같은 과정을 거쳐서 작성되었다고 가정합니다.

1. 문서에 사용할 단어의 개수 N을 정합니다.
   Ex) 5개의 단어를 정하였습니다.
2. 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정합니다.
   Ex) 위 예제와 같이 토픽이 2개라고 하였을 때 강아지 토픽을 60%, 과일 토픽을 40%와 같이 선택할 수 있습니다.
3. 문서에 사용할 각 단어를 (아래와 같이) 정합니다.
  - 3.1 토픽 분포에서 토픽 T를 확률적으로 고릅니다.
    Ex) 60% 확률로 강아지 토픽을 선택하고, 40% 확률로 과일 토픽을 선택할 수 있습니다.
  - 3.2. 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고릅니다.
    Ex) 강아지 토픽을 선택하였다면, 33% 확률로 강아지란 단어를 선택할 수 있습니다. 이제 3)을 반복하면서 문서를 완성합니다.

이러한 과정을 통해 문서가 작성되었다는 가정 하에 LDA는 토픽을 뽑아내기 위하여 위 과정을 역으로 추적하는 역공학(reverse engneering)을 수행합니다.

### 3. LDA의 수행하기
이제 LDA의 수행 과정을 정리해보겠습니다.

1) 사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.
앞서 말하였듯이 LDA에게 토픽의 개수를 알려주는 역할은 사용자의 역할입니다. LDA는 토픽의 개수 k를 입력받으면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정합니다.

2) 모든 단어를 k개 중 하나의 토픽에 할당합니다.
이제 LDA는 모든 문서의 모든 단어에 대해서 k개 중 하나의 토픽을 랜덤으로 할당합니다. 이 작업이 끝나면 각 문서는 토픽을 가지며, 토픽은 단어 분포를 가지는 상태입니다. 물론 랜덤으로 할당하였기 때문에 사실 이 결과는 전부 틀린 상태입니다. 만약 한 단어가 한 문서에서 2회 이상 등장하였다면, 각 단어는 서로 다른 토픽에 할당되었을 수도 있습니다.

3) 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행합니다. (iterative)

3-1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당됩니다.
- p(topic t | document d) : 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
- p(word w | topic t) : 단어 w를 갖고 있는 모든 문서들 중 토픽 t가 할당된 비율

이를 반복하면, 모든 할당이 완료된 수렴 상태가 됩니다. 두 가지 기준이 어떤 의미인지 예를 들어보겠습니다. 설명의 편의를 위해서 두 개의 문서라는 새로운 예를 사용합니다.

....

....

....

### 4. 잠재 디리클레 할당과 잠재 의미 분석의 차이

LSA : DTM을 차원 축소 하여 **축소 차원**에서 근접 단어들을 토픽으로 묶는다.  
LDA : 단어가 특정 토픽에 존재할 확률과 문서에 **특정 토픽이 존재할 확률을 결합확률로 추정**하여 토픽을 추출한다.

### 5. 실습을 통한 이해
이제 LDA를 실습을 통해 직접 진행해보도록 하겠습니다. LSA 챕터에서는 사이킷런(sklearn)을 사용하였지만, 이번 챕터에서는 gensim을 사용하므로 앞서 챕터와 실습 과정이 확연히 다릅니다. 사이킷런을 통해 LDA를 진행하는 실습은 아래의 링크에 작성하였습니다. 아래의 링크에서는 LSA 챕터의 실습과 진행 과정이 거의 유사하니 참고하시기 바랍니다.

사이킷런으로 LDA 실습 : https://wikidocs.net/40710

##### 1) 정수 인코딩과 단어 집합 만들기
바로 이전 챕터인 LSA 챕터에서 사용하였던 Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 뉴스 데이터를 다시 사용합니다. 전처리 과정은 이전 챕터와 중복되므로 생략합니다. 동일한 전처리 과정을 거친 후에 tokenized_doc으로 저장한 상태라고 합시다. 훈련용 뉴스를 5개만 출력해보겠습니다.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footers','quotes'))
documents = dataset.data

news_df = pd.DataFrame({'document':documents})

# Cleansing
# Remove special characters
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")

# Remove words less than 3 in length
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# Lowercase transformation for all words.
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

#  Stopword extratcion
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

이제 각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수를 기록해보겠습니다. 여기서는 각 단어를 (word_id, word_frequency)의 형태로 바꾸고자 합니다.  
- word_id는 단어가 정수 인코딩된 값이고, 
- word_frequency는 해당 뉴스에서의 해당 단어의 빈도수를 의미합니다. 

이는 gensim의 corpora.Dictionary()를 사용하여 손쉽게 구할 수 있습니다. 전체 뉴스에 대해서 정수 인코딩을 수행하고, 두번째 뉴스를 출력해봅시다.

In [2]:
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)   # Build dictionay 
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]   # Build BoW

print(corpus[1])


[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


두번째 뉴스의 출력 결과를 봅시다. 위의 출력 결과 중에서 (66, 2)는 정수 인코딩이 66으로 할당된 단어가 두번째 뉴스에서는 두 번 등장하였음을 의미합니다. 66이라는 값을 가지는 단어가 정수 인코딩이 되기 전에는 어떤 단어였는지 확인하여봅시다. 이는 dictionary[]에 기존 단어가 무엇인지 알고자하는 정수값을 입력하여 확인할 수 있습니다.

In [3]:
print(dictionary[66])

faith


기존에는 단어 'faith'이었음을 알 수 있습니다. 총 학습된 단어의 개수를 확인해보겠습니다. 이는 dictionary의 길이를 확인하면 됩니다.

In [4]:
len(dictionary)

64281

##### 2) LDA 모델 훈련시키기
기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있었으므로 토픽의 개수를 20으로 하여 LDA 모델을 학습시켜보도록 하겠습니다.

In [5]:
import gensim

NUM_TOPICS = 20 
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)

for topic in topics:
    print(topic)

(0, '0.017*"radar" + 0.015*"disease" + 0.013*"candida" + 0.010*"detector"')
(1, '0.018*"chip" + 0.016*"scsi" + 0.014*"drive" + 0.012*"encryption"')
(2, '0.033*"armenian" + 0.029*"armenians" + 0.023*"turkish" + 0.011*"armenia"')
(3, '0.013*"information" + 0.009*"public" + 0.008*"mail" + 0.007*"university"')
(4, '0.041*"file" + 0.020*"program" + 0.017*"output" + 0.017*"window"')
(5, '0.015*"windows" + 0.012*"thanks" + 0.010*"anyone" + 0.010*"know"')
(6, '0.013*"people" + 0.011*"would" + 0.009*"said" + 0.007*"know"')
(7, '0.027*"jesus" + 0.017*"christian" + 0.016*"bible" + 0.014*"church"')
(8, '0.015*"would" + 0.009*"people" + 0.008*"think" + 0.008*"like"')
(9, '0.018*"april" + 0.015*"chicago" + 0.012*"york" + 0.011*"lost"')
(10, '0.016*"period" + 0.014*"play" + 0.009*"power" + 0.008*"pittsburgh"')
(11, '0.022*"available" + 0.016*"image" + 0.012*"version" + 0.012*"graphics"')
(12, '0.011*"like" + 0.009*"time" + 0.008*"back" + 0.008*"would"')
(13, '0.021*"price" + 0.016*"sale" + 0.014*"off

각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줍니다. 또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 20개의 토픽은 0부터 19까지의 번호가 할당되어져 있습니다. passes는 알고리즘의 동작 횟수를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 됩니다. 여기서는 총 15회를 수행하였습니다. 여기서는 num_words=4로 총 4개의 단어만 출력하도록 하였습니다. 만약 10개의 단어를 출력하고 싶다면 아래의 코드를 수행하면 됩니다.

In [6]:
print(ldamodel.print_topics())

[(0, '0.017*"radar" + 0.015*"disease" + 0.013*"candida" + 0.010*"detector" + 0.010*"infections" + 0.010*"infection" + 0.009*"illness" + 0.009*"cells" + 0.008*"plate" + 0.007*"mydisplay"'), (1, '0.018*"chip" + 0.016*"scsi" + 0.014*"drive" + 0.012*"encryption" + 0.011*"data" + 0.011*"keys" + 0.010*"system" + 0.010*"clipper" + 0.008*"used" + 0.007*"disk"'), (2, '0.033*"armenian" + 0.029*"armenians" + 0.023*"turkish" + 0.011*"armenia" + 0.011*"russian" + 0.011*"turks" + 0.010*"genocide" + 0.009*"muslim" + 0.009*"azerbaijan" + 0.008*"soviet"'), (3, '0.013*"information" + 0.009*"public" + 0.008*"mail" + 0.007*"university" + 0.006*"internet" + 0.006*"send" + 0.006*"list" + 0.006*"health" + 0.005*"news" + 0.005*"research"'), (4, '0.041*"file" + 0.020*"program" + 0.017*"output" + 0.017*"window" + 0.017*"entry" + 0.013*"files" + 0.010*"line" + 0.009*"widget" + 0.008*"section" + 0.008*"build"'), (5, '0.015*"windows" + 0.012*"thanks" + 0.010*"anyone" + 0.010*"know" + 0.009*"system" + 0.009*"like" 

##### 3) LDA 시각화 하기
LDA 시각화를 위해서는 pyLDAvis의 설치가 필요합니다. 윈도우의 명령 프롬프트나 MAC/UNIX의 터미널에서 아래의 명령을 수행하여 pyLDAvis를 설치하시기 바랍니다.

In [7]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


좌측의 원들은 각각의 20개의 토픽을 나타냅니다. 
- 각 원과의 거리는 각 토픽들이 서로 얼마나 다른지를 보여줍니다. 
- 만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미입니다. 
- 위의 그림에서는 10번 토픽을 클릭하였고, 이에 따라 우측에는 10번 토픽에 대한 정보가 나타납니다. 

한 가지 주의할 점은 LDA 모델의 출력 결과에서는 토픽 번호가 0부터 할당되어 0~19의 숫자가 사용된 것과는 달리 위의 LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽 번호는 이제 +1이 된 값인 1~20까지의 값을 가집니다.

##### 4) 문서 별 토픽 분포 보기
위에서 토픽 별 단어 분포는 확인하였으나, 아직 문서 별 토픽 분포에 대해서는 확인하지 못 하였습니다.  

우선 문서 별 토픽 분포를 확인하는 방법을 보겠습니다. 각 문서의 토픽 분포는 이미 훈련된 LDA 모델인 ldamodel[]에 전체 데이터가 정수 인코딩 된 결과를 넣은 후에 확인이 가능합니다. 여기서는 책의 지면의 한계로 상위 5개의 문서에 대해서만 토픽 분포를 확인해보겠습니다.

In [8]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i == 5:
        break
    print(i, '번째 문서의 topic 비율은 ', topic_list)

0 번째 문서의 topic 비율은  [(2, 0.026458984), (6, 0.3032086), (8, 0.1586899), (14, 0.025988486), (18, 0.34685472), (19, 0.12731141)]
1 번째 문서의 topic 비율은  [(7, 0.27460337), (8, 0.6338896), (14, 0.06912228)]
2 번째 문서의 topic 비율은  [(1, 0.019899106), (6, 0.43324715), (8, 0.24624489), (18, 0.2874664)]
3 번째 문서의 topic 비율은  [(0, 0.015245813), (1, 0.3101055), (8, 0.29681563), (12, 0.365295)]
4 번째 문서의 topic 비율은  [(4, 0.1221116), (14, 0.8445457)]


위의 출력 결과에서 (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미합니다. 예를 들어 0번째 문서의 토픽 비율에서 (9, 0.15796122)은 9번 토픽이 15%의 분포도를 가지는 것을 의미합니다. 위의 코드를 응용하여 좀 더 깔끔한 형태인 데이터프레임 형식으로 출력해보겠습니다.

In [9]:
def make_topic_table_per_doc(ldamodel, corpus, texts):
    topic_table = pd.DataFrame()
    
    # Take out the document number and the portion of the topic in the docuement one  line  at a time
    for i, topic_list in enumerate(ldamodel[corpus]):
        ''' if ldamodel.per_word_topics:
                topic_list[0]
            else:
                topic_list
            ==> topic_list[0] if ldamodel.per_word_topics else topic_list '''
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key = lambda x: (x[1]), reverse = True)
        
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.
        
        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j ==0:   # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index = True)
                
             # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.                
            else:
                break
    return(topic_table)  

In [10]:
topic_table = make_topic_table_per_doc(ldamodel, corpus, tokenized_doc)
topic_table = topic_table.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topic_table.columns = ['Doc#', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topic_table[:10]

,Doc#,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,18.0,0.3469,"[(2, 0.026456153), (6, 0.30326158), (8, 0.1586..."
1,1,8.0,0.6341,"[(7, 0.27459067), (8, 0.634059), (14, 0.068965..."
2,2,6.0,0.4332,"[(1, 0.019898877), (6, 0.43319672), (8, 0.2463..."
3,3,12.0,0.3654,"[(0, 0.01524581), (1, 0.3101011), (8, 0.296740..."
4,4,14.0,0.8445,"[(4, 0.12211288), (14, 0.8445444)]"
5,5,6.0,0.3090,"[(1, 0.20827271), (6, 0.3090293), (7, 0.217192..."
6,6,5.0,0.4000,"[(1, 0.022554861), (5, 0.40000802), (8, 0.2496..."
7,7,8.0,0.3583,"[(4, 0.11807816), (6, 0.32899383), (7, 0.02586..."
8,8,3.0,0.4433,"[(3, 0.44328827), (6, 0.42333704), (15, 0.1083..."
9,9,12.0,0.3742,"[(5, 0.052028354), (8, 0.3319804), (12, 0.3741..."


http://s-space.snu.ac.kr/bitstream/10371/95582/1/22_1_%EB%82%A8%EC%B6%98%ED%98%B8.pdf  
https://bab2min.tistory.com/568  
https://annalyzin.wordpress.com/2015/06/21/laymans-explanation-of-topic-modeling-with-lda-2/  
https://towardsdatascience.com/latent-dirichlet-allocation-15800c852699  
https://radimrehurek.com/gensim/wiki.html#latent-dirichlet-allocation  
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/  

모델 저장 및 로드 하기 : https://stackabuse.com/python-for-nlp-working-with-the-gensim-library-part-2/

전반적으로 참고하기 좋은 글 : https://shichaoji.com/2017/04/25/topicmodeling/

동영상 강의 : https://blog.naver.com/chunjein/220946362463

뉴스를 가지고 할 수 있는 다양한 일들 : https://www.slideshare.net/koorukuroo/20160813-pycon2016apac